In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys 
sys.path.insert(0, "../src")
import configparser
import pandas as pd 

from utils.dice_score import multiclass_dice_coeff
import numpy as np
import torch
import matplotlib.pyplot as plt
from os.path import join as oj
import my_data
pd.set_option('display.float_format', lambda x: '%.3f' % x)
cuda =  torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")
config = configparser.ConfigParser()
config.read('../config.ini');
data_path =config['DATASET']['data_path'] 


In [ ]:
x_full,y, _, _ = my_data.load_layer_data(oj(data_path, 'lno'))
x_full = x_full[:-1]
x_full.size

In [ ]:
x,y, _, _ = my_data.load_layer_data(oj(data_path, 'lno_halfHour'))
x.size

In [ ]:
(y!=255).mean() *x.size 

In [ ]:
((y!=255).mean() *x.size + 0.02*x_full.size)

In [ ]:
x_rough,y_rough, _, _ = my_data.load_layer_data(oj(data_path, 'lno_halfHour'))
# how many pixels labelled in rough annot
print((y_rough == 255).mean())
del x_rough
del y_rough

In [4]:
y_second_formatted = y_second_formatted[1024:, :1024] # only one quarter annotated
y = y[-1,1024:, :1024]

In [5]:
y_pred = y_second_formatted[None,:]
y_pred_one_hot = torch.nn.functional.one_hot(torch.Tensor(y_pred).to(torch.int64), 
                                            num_classes=3).permute(0, 3, 1, 2).squeeze()[None, :]
dice = multiclass_dice_coeff(y_pred_one_hot.float(), 
                    torch.Tensor(y[None,:]), 
                    num_classes=3,separated_up = True)


In [ ]:
dice.mean() # all classes

In [ ]:
dice

In [ ]:
dice[1:].mean() # separated up

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize= (12,4))
axes[0].imshow(x[-1, 0,1024:, :1024], cmap = plt.get_cmap('gray')  )
axes[1].imshow(y)
# add title
axes[1].set_title('First annotation', fontsize = 20)
axes[2].imshow(y_second_formatted)

axes[2].set_title('Second annotation', fontsize = 20)
# axes[3].imshow(y != y_second_formatted)
# # remove the ticks on x and y
# axes[3].set_title('Difference')
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])

#have a red box in the image
rect = plt.Rectangle((250, 550), 200, 200, edgecolor='r', facecolor='none', linewidth=2)
axes[2].add_patch(rect)
rect = plt.Rectangle((250, 550), 200, 200, edgecolor='r', facecolor='none', linewidth=2)
axes[1].add_patch(rect)



# axes[3].bar(np.arange(2), [entropy[y == y_second_formatted].mean(), entropy[y != y_second_formatted].mean()],color=['green', 'red'] )
# axes[3].set_xticks(np.arange(2))
# # make arrows to the plots

# axes[3].set_xticklabels(['Annotations match', 'Do not match'], rotation=10, fontsize = 15)
# axes[3].set_ylabel('Entropy (NN)', fontsize = 15);
# axes[3].set_yticks([0, 0.3, .6])
plt.tight_layout()
fig_path =config['PATHS']['figure_path'] 

plt.savefig(oj(fig_path,'lno_annotations.png'), bbox_inches='tight', dpi=300)

In [12]:
# load a model and

from scipy.signal import convolve2d
# from unet import UNet
import segmentation_models_pytorch as smp
# from tqdm.notebook import  tqdm

net =  smp.UnetPlusPlus(
                        encoder_name="resnet34",
                        encoder_weights="imagenet",
                        in_channels=1,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
                        classes=3  # model output channels (number of classes in your dataset)
                    ).to(device=device)

net.load_state_dict(torch.load(oj(config['PATHS']['model_path'] , '0073024568' +  ".pt" )))
y_pred = net(torch.Tensor(x[-1:, 0,1024:, :1024]/255.0)[None, :].to(device))
y_pred_softmax = torch.nn.functional.softmax(y_pred[0], dim=0).detach().cpu().numpy()
entropy = ( - (y_pred_softmax * np.log(y_pred_softmax)).sum(axis=0))

In [13]:

from scipy.signal import convolve2d
add_val = 32
mean_arr= np.ones((add_val,add_val))/np.square(add_val)


correct_vals = convolve2d((y != y_second_formatted).astype(float), mean_arr, mode = 'same')[::add_val, ::add_val].reshape(-1)
entropy_smooth = convolve2d(entropy, mean_arr, mode = 'same')[::add_val, ::add_val].reshape(-1)

In [ ]:


vals = [x*100 for x in [ .1, .2, .3, .4,.5,.6,.7,.8,.9, .95, .99,  .999]]
# vals 

correct_mean = []
for i in range(len(vals)):

    correct_mean.append(correct_vals[np.where((entropy_smooth > np.percentile(entropy_smooth, vals[i])) )].mean())
plt.plot(vals, np.asarray(correct_mean)*100, marker = 'o')


ninetyfive = correct_vals[np.where((entropy_smooth > np.percentile(entropy_smooth, 95)) )].mean()*100
plt.plot([95, ], [ninetyfive,], marker = 'o', c= 'r', label = "95%",)
plt.xlabel('Percentile of entropy', fontsize = 15)

plt.ylabel('% Wrong pixels', fontsize = 15)
plt.legend()
# plt.savefig(oj(fig_path,'entropy_corrected.png'), bbox_inches='tight', dpi=300)



